In [1]:
#Make it run on Colab, can be time-consuming depending on your machine

import numpy as np 
import tensorflow as tf 
from tensorflow import keras 

from keras.utils import to_categorical

N = 10000 
planes = 31 
moves = 361

input_data = np.random.randint(2, size=(N, 19, 19, planes)) #what is on the board
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,)) #what to play 
policy = to_categorical (policy)

value = np.random.randint(2, size=(N,)) #proba of whites to win
value = value.astype ('float32')

In [30]:
from keras import layers

input = keras.Input(shape=(19, 19, planes), name='board')

x = layers.Conv2D(32, 3, activation='relu', padding='same')(input)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)

policy_head = layers.Conv2D(1, 3, activation='relu', padding='same')(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Dense(moves, activation='softmax', name='policy')(policy_head) 

value_head = layers.Flatten()(x)
value_head = layers.Dense(1, activation='sigmoid', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head]) 
model.summary ()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 board (InputLayer)             [(None, 19, 19, 31)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 19, 19, 32)   8960        ['board[0][0]']                  
                                                                                                  
 conv2d_1 (Conv2D)              (None, 19, 19, 32)   9248        ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 19, 19, 32)   9248        ['conv2d_1[0][0]']           

2022-10-14 15:05:17.268914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


__________________________________________________________________________________________________


In [31]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.001), loss={'value': 'mse', 'policy': 'categorical_crossentropy'})
model.fit(input_data, {'policy': policy, 'value': value}, epochs=20, batch_size=128, validation_split=0.1)
model.save ('test.h5')

Epoch 1/20


/Users/GuillaumeKunsch/opt/anaconda3/envs/CoursIASDsurDL/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


 40/704 [>.............................] - ETA: 2:20 - loss: 6.1392 - policy_loss: 5.8890 - value_loss: 0.2502

KeyboardInterrupt: 